# Multiprocessing

In [7]:
cd optimizing/multiprocessing/

/Volumes/HDD-Data/PycharmProjects/AdvancedPython/source/optimizing/multiprocessing


In [15]:
# %load processes_pi.py
# file: processes_pi.py

"""Calculation of pi with Monte Carlo and multiprocessing.
"""

from __future__ import print_function

import math
try:
    from multiprocessing import Process, Queue                  #1
except ImportError:
    from processing import Process, Queue                       #2
import random
import timeit
import sys


if sys.version_info[0] < 3:
    range = xrange

def count_inside(total):                                        #3
    """Count the hits inside the circle.

    This function will be run multiple times in different
    processes.
    """
    inside_count = 0
    for _ in range(total):
        x = random.random()
        y = random.random()
        dist = math.sqrt(x * x + y * y)
        if dist < 1:
            inside_count += 1
    return inside_count


def calc_pi(total):                                             #4
    """Determine pi _without_ multprocessing as refrence.
    """
    return 4 * count_inside(total) / float(total)


def count_inside_process(queue, total):                         #5
    """Count the hits inside the circle in a seperate process.
    """
    queue.put(count_inside(total))


def calc_pi_processes(total, process_count):                    #6
    """Calculate pi spread of several processses.

    We need to split the task into sub tasks before we can hand
    them to other processes.
    """
    min_n = total // process_count                              #7
    counters = [min_n] * process_count                          #8
    reminder = total % process_count                            #9
    for count in range(reminder):                               #10
        counters[count] += 1                                    #11
    queues_processes = []                                       #12
    for counter in counters:                                    #13
        queue = Queue()                                         #14
        process = Process(target=count_inside_process,
                           args=(queue, counter))               #15
        process.start()                                         #16
        queues_processes.append((queue, process))               #17
    inside_count = sum(queue.get() for queue, _
                       in queues_processes)                     #18
    for _, process in queues_processes:                         #19
        process.join()                                          #20
    return 4 * inside_count / float(total)                      #21

if __name__ == '__main__':

    def test():
        """Check if it works.
        """
        process_count = 3                                       #22
        total = int(1e6)                                        #23
        print('number of tries: %2.0e' % total)                 #24

        start = timeit.default_timer()                          #25
        pi = calc_pi(total)
        one_time = timeit.default_timer() - start
        print('pi:', pi)
        print('run time one process:', one_time)

        start = timeit.default_timer()                          #26
        pi = calc_pi_processes(total, process_count)            #27
        two_time = timeit.default_timer() - start
        print('pi:', pi)
        print('run time %d processes:' % process_count, two_time)

        print('ratio:', one_time / two_time)                     #28
        print('diff:', two_time - one_time)

    test()


number of tries: 1e+06
pi: 3.142568
run time one process: 1.0330762279999703
pi: 3.139288
run time 3 processes: 0.6635737309999854
ratio: 1.5568371376654044
diff: -0.36950249699998494


In [21]:
# %load pool_pi.py
# file pool_pi.py

"""Multiprocessing with a pool of workers.
"""

from __future__ import print_function

import math
try:
    from multiprocessing import Pool, cpu_count                  #1
except ImportError:
    from processing import Pool, cpu_count                       #2
import random
import timeit
import sys

if sys.version_info[0] < 3:
    range = xrange

def count_inside(total):
    """Count the hits inside the circle.

    This function will be run multiple times in different
    processes.
    """
    inside_count = 0
    for _ in range(total):
        x = random.random()
        y = random.random()
        dist = math.sqrt(x * x + y * y)
        if dist < 1:
            inside_count += 1
    return inside_count


def calc_pi(total):
    """Determine pi _without_ multprocessing as refrence.
    """
    return 4 * count_inside(total) / float(total)


def calc_pi_workers(total, workers=None):                           #3
    """Calculate pi spread of several processses with workers.

    We need to split the task into sub tasks before we can hand
    them to other process that run workers
    """
    if not workers:
        workers = cpu_count()                                  #4
    min_n = total // workers
    counters = [min_n] * workers
    reminder = total % workers
    for count in range(reminder):
        counters[count] += 1
    pool = Pool(processes=workers)                              #5
    results = [pool.apply_async(count_inside, (counter,))
                       for counter in counters]                 #6
    inside_count = sum(result.get() for result in results)      #7
    return 4 * inside_count / float(total)                      #8

if __name__ == '__main__':

    def test():
        """Check if it works.
        """
        workers = 2

        total = int(1e7)

        print('number of tries: %2.0e' % total)
        start = timeit.default_timer()
        pi = calc_pi(total)
        no_time = timeit.default_timer() - start
        print('pi:', pi)
        print('run time no workers:', no_time)

        start = timeit.default_timer()
        pi = calc_pi_workers(total, workers)
        two_time = timeit.default_timer() - start
        print('pi:', pi)
        print('run time %d workers:' % workers, two_time)
        print('ratio:', no_time / two_time)
        print('diff:', two_time - no_time)

    test()


number of tries: 1e+07
pi: 3.142324
run time no workers: 9.153412079000077
pi: 3.1409484
run time 2 workers: 4.915238617
ratio: 1.8622518238162022
diff: -4.238173462000077
